<a href="https://colab.research.google.com/github/harrymkwn/InfluenceAnalysis/blob/master/AllEmointGen_Amp5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from google.colab import  drive
drive.mount('/content/drive')
import nltk
from nltk.corpus import stopwords
nltk.download('all')
device_name = tf.test.gpu_device_name()
print(device_name)
df = pd.read_csv('/content/drive/MyDrive/InfluenceAnalysis/Data/Tweets_clean_data.csv')
df = df.sample(frac=1,random_state=32)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading pack

In [2]:
category_dict = {"anger":[1,0,0,0],"sadness":[0,1,0,0],"fear":[0,0,1,0],"joy":[0,0,0,1]}
data_tweet = [x.lower().split() for x in df['Tweet']]
data_cat = np.array([category_dict[x] for x in df['Emotion']])


In [3]:
print(data_tweet[:5])
print(data_cat[:5])

[['with', 'their', 'faggy', 'colors', 'are', 'nice', 'is', 'ok', 'too', 'even', 'tho', 'some', 'might', 'take', 'offense', 'because', 'words', 'lol'], ['unbelievable', 'takes', '10', 'minutes', 'to', 'get', 'through', 'to', 'then', 'there', 'is', 'a', 'fault', 'and', 'the', 'call', 'hangs', 'up', 'treatcustomersfairly'], ['well', 'i', 'did', 'hear', 'once', 'before', 'that', 'girls', 'are', 'attracted', 'to', 'men', 'that', 'lok', 'like', 'their', 'dad', 'ok', 'hand'], ['agreed', 'so', 'tired', 'of', 'this', 'nonsense', 'soros', 'must', 'be', 'elated'], ['by', 'the', 'way', 'i', 'am', 'wearing', 'the', 'smile', 'you', 'gave', 'me', 'today', 'n', 'you', 'me']]
[[1 0 0 0]
 [1 0 0 0]
 [0 1 0 0]
 [0 0 0 1]
 [0 0 0 1]]


In [4]:
# Parameters
Min_count = 0
Embedding_size = 200
Window_size = 5
Negative_sampling = 00

In [5]:
w2v_sg = Word2Vec(min_count=Min_count,
                     window=Window_size,
                     size=Embedding_size,
                     negative=Negative_sampling,sg=1)
w2v_sg.build_vocab(data_tweet)
w2v_sg.train(data_tweet, total_examples=w2v_sg.corpus_count, epochs=5)
w2v_cbow = Word2Vec(min_count=Min_count,
                     window=Window_size,
                     size=Embedding_size,
                     negative=Negative_sampling,sg=0)
w2v_cbow.build_vocab(data_tweet)
w2v_cbow.train(data_tweet, total_examples=w2v_cbow.corpus_count, epochs=5)
w2v_sg.wv.init_sims(True)
w2v_cbow.wv.init_sims(True)

In [6]:
vocab_sg = w2v_sg.wv.vocab
vocab_sg = [x for x in vocab_sg]
vocab_cbow = w2v_cbow.wv.vocab
vocab_cbow = [x for x in vocab_cbow] 
vocab_glove = {}
with open("/content/drive/My Drive/InfluenceAnalysis/glove/glove.twitter.27B.200d.txt", 'r', encoding="utf-8") as f:
  for line in f:
      values = line.split()
      word = values[0]
      vector = np.asarray(values[1:], "float32")
      vocab_glove[word] = vector

In [7]:
batch_size = 100
Max_input_size = max([len(x) for x in data_tweet])
data_tweet = [nltk.pos_tag(x) for x in data_tweet]

In [8]:
def vec_gen(w2v,vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size,mul_factor=5):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      res=[]
      for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]:
        tweet = [(w2v.wv.word_vec(i[0]),i[1]) for i in x if i[0] in vocab]
        l=len(tweet)
        for i in range(l):
          if tweet[i][1][:2]=='JJ' or tweet[i][1][:2]=='RB' or tweet[i][1][:2]=='VB':
            tweet[i]=((tweet[i][0])*mul_factor,tweet[i][1])
        res+=[np.array([x[0] for x in tweet])]
      temp = np.array([np.pad(z.flatten(),(0,Max_input_size*Embedding_size-len(z.flatten()))).reshape(Max_input_size,Embedding_size) for z in res])
      tempres = data_cat[k*(batch_size):(k+1)*(batch_size)]
      yield (temp,tempres)          
      
def glove_gen(vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size,mul_factor=5):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      res=[]
      for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]:
        tweet = [(vocab[i[0]],i[1]) for i in x if i[0] in vocab.keys()]
        l=len(tweet)
        for i in range(l):
          if tweet[i][1][:2]=='JJ' or tweet[i][1][:2]=='RB' or tweet[i][1][:2]=='VB':
            tweet[i]=((tweet[i][0])*mul_factor,tweet[i][1])
        res+=[np.array([x[0] for x in tweet])]
      temp = np.array([np.pad(z.flatten(),(0,Max_input_size*Embedding_size-len(z.flatten()))).reshape(Max_input_size,Embedding_size) for z in res])
      tempres = data_cat[k*(batch_size):(k+1)*(batch_size)]
      yield (temp,tempres)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(data_tweet, data_cat, test_size=0.20, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.20, random_state=42)
result_table = [0,0,0,0,0,0,0,0,0]

In [10]:
epochs = 60

In [11]:
from keras import backend as K
def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recalls = true_positives / (possible_positives + K.epsilon())
        return recalls
def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precisions = true_positives / (predicted_positives + K.epsilon())
        return precisions
def f1(y_true, y_pred):
    precisions = precision(y_true, y_pred)
    recalls = recall(y_true, y_pred)
    return 2*((precisions*recalls)/(precisions+recalls+K.epsilon()))

In [12]:
model_sg = tf.keras.Sequential()
model_sg.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg.add(tf.keras.layers.LSTM(64))
model_sg.add(tf.keras.layers.Dropout(0.2))
model_sg.add(tf.keras.layers.Dense(64,activation='relu'))
model_sg.add(tf.keras.layers.Dropout(0.2))
model_sg.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy',f1,precision,recall])
sg_train_gen=vec_gen(w2v_sg,vocab_sg,X_train,y_train,batch_size,Max_input_size,Embedding_size)
sg_val_gen=vec_gen(w2v_sg,vocab_sg,X_val,y_val,batch_size,Max_input_size,Embedding_size)
history_sg_lstm = model_sg.fit_generator(sg_train_gen,validation_data=sg_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_sg.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[0]=history[1]
print("SG_LSTM")


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/60
46/46 [==============================] - 36s 131ms/step - loss: 1.3828 - accuracy: 0.2866 - f1: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.3742 - val_accuracy: 0.3300 - val_f1: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/60
46/46 [==============================] - 5s 114ms/step - loss: 1.3779 - accuracy: 0.3055 - f1: 7.6630e-05 - precision: 5.3187e-04 - recall: 4.1731e-05 - val_loss: 1.3379 - val_accuracy: 0.3567 - val_f1: 0.0878 - val_precision: 0.6290 - val_recall: 0.0475
Epoch 3/60
46/46 [==============================] - 5s 114ms/step - loss: 1.3465 - accuracy: 0.3429 - f1: 0.0499 - precision: 0.3260 - recall: 0.0284 - val_loss: 1.3040 - val_accuracy: 0.3558 - val_f1: 0.2208 - val_precision: 0.5996 - val_recall: 0.1358
Epoch 4/60
46/46 [==============================] - 5s 114ms/step - loss: 1.2777 - accuracy: 0.3967 - f1: 0.2193 - precision: 0.6892 - recall: 0.1336 - val_loss: 1.2763 - val_accuracy: 0.3850 - val_f1:

In [13]:
model_sg_bi = tf.keras.Sequential()
model_sg_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64),merge_mode='concat'))
model_sg_bi.add(tf.keras.layers.Dropout(0.2))
model_sg_bi.add(tf.keras.layers.Dense(64,activation='relu'))
model_sg_bi.add(tf.keras.layers.Dropout(0.2))
model_sg_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy',f1,precision,recall])
history_sg_bi = model_sg_bi.fit_generator(sg_train_gen,validation_data=sg_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_sg_bi.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[1]=history[1]
print("SG_BI")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/60
46/46 [==============================] - 10s 143ms/step - loss: 1.3806 - accuracy: 0.2899 - f1: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.3544 - val_accuracy: 0.3383 - val_f1: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/60
46/46 [==============================] - 6s 123ms/step - loss: 1.3348 - accuracy: 0.3550 - f1: 0.0097 - precision: 0.1886 - recall: 0.0051 - val_loss: 1.2943 - val_accuracy: 0.3833 - val_f1: 0.0969 - val_precision: 0.6508 - val_recall: 0.0525
Epoch 3/60
46/46 [==============================] - 6s 122ms/step - loss: 1.2310 - accuracy: 0.4549 - f1: 0.1924 - precision: 0.6820 - recall: 0.1147 - val_loss: 1.2450 - val_accuracy: 0.4400 - val_f1: 0.2586 - val_precision: 0.6039 - val_recall: 0.1658
Epoch 4/60
46/46 [==============================] - 6s 122ms/step - loss: 1.1167 - accuracy: 0.5252 - f1: 0.3648 - precision: 0.7206 - recall: 0.2464 - val_loss: 1.1867 - val_accuracy: 0.4950 - val_f1: 0.3596 - va

In [14]:
model_sg_rnn = tf.keras.Sequential()
model_sg_rnn.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg_rnn.add(tf.keras.layers.SimpleRNN(64))
model_sg_rnn.add(tf.keras.layers.Dropout(0.2))
model_sg_rnn.add(tf.keras.layers.Dense(64,activation='relu'))
model_sg_rnn.add(tf.keras.layers.Dropout(0.2))
model_sg_rnn.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy',f1,precision,recall])
history_sg_rnn = model_sg_rnn.fit_generator(sg_train_gen,validation_data=sg_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_sg_rnn.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)


result_table[2]=history[1]
print("SG_RNN")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/60
46/46 [==============================] - 9s 165ms/step - loss: 1.3877 - accuracy: 0.2759 - f1: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.3701 - val_accuracy: 0.3317 - val_f1: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/60
46/46 [==============================] - 7s 164ms/step - loss: 1.3807 - accuracy: 0.3036 - f1: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.3631 - val_accuracy: 0.3417 - val_f1: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/60
46/46 [==============================] - 7s 159ms/step - loss: 1.3752 - accuracy: 0.3064 - f1: 7.2363e-05 - precision: 9.4072e-04 - recall: 3.7629e-05 - val_loss: 1.3608 - val_accuracy: 0.3400 - val_f1: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/60
46/46 [==============================] - 7s 158ms/step - loss: 1.3693 - accuracy: 0.3263 - f1: 0.0178 - precision: 0.1920 - recall: 0.0097 - val_loss: 1.36

In [15]:
model_cbow = tf.keras.Sequential()
model_cbow.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow.add(tf.keras.layers.LSTM(64))
model_cbow.add(tf.keras.layers.Dropout(0.2))
model_cbow.add(tf.keras.layers.Dense(64,activation='relu'))
model_cbow.add(tf.keras.layers.Dropout(0.2))
model_cbow.add(tf.keras.layers.Dense(4, activation='softmax'))
model_cbow.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy',f1,precision,recall])
cbow_train_gen=vec_gen(w2v_cbow,vocab_cbow,X_train,y_train,batch_size,Max_input_size,Embedding_size)
cbow_val_gen=vec_gen(w2v_cbow,vocab_cbow,X_val,y_val,batch_size,Max_input_size,Embedding_size)
history_cbow_lstm = model_cbow.fit_generator(cbow_train_gen,validation_data=cbow_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_cbow.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[3]=history[1]
print("CBOW_LSTM")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/60
46/46 [==============================] - 8s 133ms/step - loss: 1.3825 - accuracy: 0.2964 - f1: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.3734 - val_accuracy: 0.3308 - val_f1: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/60
46/46 [==============================] - 5s 118ms/step - loss: 1.3758 - accuracy: 0.3067 - f1: 0.0082 - precision: 0.0497 - recall: 0.0048 - val_loss: 1.3373 - val_accuracy: 0.3617 - val_f1: 0.0464 - val_precision: 0.6379 - val_recall: 0.0242
Epoch 3/60
46/46 [==============================] - 5s 118ms/step - loss: 1.3336 - accuracy: 0.3497 - f1: 0.1107 - precision: 0.6338 - recall: 0.0636 - val_loss: 1.3317 - val_accuracy: 0.3600 - val_f1: 0.2415 - val_precision: 0.5526 - val_recall: 0.1550
Epoch 4/60
46/46 [==============================] - 5s 118ms/step - loss: 1.2873 - accuracy: 0.3838 - f1: 0.2080 - precision: 0.6945 - recall: 0.1265 - val_loss: 1.2957 - val_accuracy: 0.3800 - val_f1: 0.1836 - val

In [16]:
model_cbow_bi = tf.keras.Sequential()
model_cbow_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64),merge_mode='concat'))
model_cbow_bi.add(tf.keras.layers.Dropout(0.2))
model_cbow_bi.add(tf.keras.layers.Dense(64,activation='relu'))
model_cbow_bi.add(tf.keras.layers.Dropout(0.2))
model_cbow_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_cbow_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy',f1,precision,recall])
history_cbow_bi = model_cbow_bi.fit_generator(cbow_train_gen,validation_data=cbow_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_cbow_bi.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[4]=history[1]
print("CBOW_BI")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/60
46/46 [==============================] - 10s 145ms/step - loss: 1.3830 - accuracy: 0.2768 - f1: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.3615 - val_accuracy: 0.3483 - val_f1: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/60
46/46 [==============================] - 6s 122ms/step - loss: 1.3434 - accuracy: 0.3524 - f1: 0.0046 - precision: 0.1196 - recall: 0.0024 - val_loss: 1.3157 - val_accuracy: 0.3900 - val_f1: 0.0526 - val_precision: 0.6917 - val_recall: 0.0275
Epoch 3/60
46/46 [==============================] - 6s 123ms/step - loss: 1.2474 - accuracy: 0.4400 - f1: 0.1695 - precision: 0.7100 - recall: 0.0998 - val_loss: 1.2385 - val_accuracy: 0.4375 - val_f1: 0.2743 - val_precision: 0.5665 - val_recall: 0.1817
Epoch 4/60
46/46 [==============================] - 6s 125ms/step - loss: 1.1136 - accuracy: 0.5286 - f1: 0.3922 - precision: 0.7217 - recall: 0.2711 - val_loss: 1.1597 - val_accuracy: 0.5008 - val_f1: 0.3910 - va

In [17]:
model_cbow_rnn = tf.keras.Sequential()
model_cbow_rnn.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow_rnn.add(tf.keras.layers.SimpleRNN(64))
model_cbow_rnn.add(tf.keras.layers.Dropout(0.2))
model_cbow_rnn.add(tf.keras.layers.Dense(64,activation='relu'))
model_cbow_rnn.add(tf.keras.layers.Dropout(0.2))
model_cbow_rnn.add(tf.keras.layers.Dense(4, activation='softmax'))
model_cbow_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy',f1,precision,recall])
history_cbow_rnn = model_cbow_rnn.fit_generator(cbow_train_gen,validation_data=cbow_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_cbow_rnn.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[5]=history[1]
print("CBOW_RNN")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/60
46/46 [==============================] - 9s 161ms/step - loss: 1.3910 - accuracy: 0.2500 - f1: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.3728 - val_accuracy: 0.3317 - val_f1: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/60
46/46 [==============================] - 7s 151ms/step - loss: 1.3800 - accuracy: 0.2954 - f1: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.3707 - val_accuracy: 0.3300 - val_f1: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/60
46/46 [==============================] - 7s 151ms/step - loss: 1.3779 - accuracy: 0.2977 - f1: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.3695 - val_accuracy: 0.3367 - val_f1: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/60
46/46 [==============================] - 7s 152ms/step - loss: 1.3724 - accuracy: 0.3098 - f1: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - va

In [18]:
model_glove = tf.keras.Sequential()
model_glove.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_glove.add(tf.keras.layers.LSTM(64))
model_glove.add(tf.keras.layers.Dropout(0.2))
model_glove.add(tf.keras.layers.Dense(64,activation='relu'))
model_glove.add(tf.keras.layers.Dropout(0.2))
model_glove.add(tf.keras.layers.Dense(4, activation='softmax'))
model_glove.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy',f1,precision,recall])
glove_train_gen=glove_gen(vocab_glove,X_train,y_train,batch_size,Max_input_size,Embedding_size)
glove_val_gen=glove_gen(vocab_glove,X_val,y_val,batch_size,Max_input_size,Embedding_size)
history_glove_lstm = model_glove.fit_generator(glove_train_gen,validation_data=glove_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_glove.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[6]=history[1]
print("GLOVE_LSTM")


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/60
46/46 [==============================] - 4s 43ms/step - loss: 1.3756 - accuracy: 0.3153 - f1: 0.0148 - precision: 0.0646 - recall: 0.0088 - val_loss: 1.2618 - val_accuracy: 0.4292 - val_f1: 0.2256 - val_precision: 0.7417 - val_recall: 0.1333
Epoch 2/60
46/46 [==============================] - 1s 30ms/step - loss: 1.2312 - accuracy: 0.4300 - f1: 0.2654 - precision: 0.6724 - recall: 0.1684 - val_loss: 1.1901 - val_accuracy: 0.4733 - val_f1: 0.4556 - val_precision: 0.6147 - val_recall: 0.3625
Epoch 3/60
46/46 [==============================] - 1s 29ms/step - loss: 1.1158 - accuracy: 0.4837 - f1: 0.3970 - precision: 0.7034 - recall: 0.2906 - val_loss: 1.0966 - val_accuracy: 0.4942 - val_f1: 0.4712 - val_precision: 0.6431 - val_recall: 0.3725
Epoch 4/60
46/46 [==============================] - 1s 30ms/step - loss: 0.9549 - accuracy: 0.5660 - f1: 0.4838 - precision: 0.7637 - recall: 0.3597 - val_loss: 1.1100 - val_accuracy: 0.5017 - val_f1: 0.4384 - val_precision: 0.6702 - val_rec

In [19]:
model_glove_bi = tf.keras.Sequential()
model_glove_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_glove_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64),merge_mode='concat'))
model_glove_bi.add(tf.keras.layers.Dropout(0.2))
model_glove_bi.add(tf.keras.layers.Dense(64,activation='relu'))
model_glove_bi.add(tf.keras.layers.Dropout(0.2))
model_glove_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_glove_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy',f1,precision,recall])
history_glove_bi = model_glove_bi.fit_generator(glove_train_gen,validation_data=glove_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_glove_bi.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[7]=history[1]
print("GLOVE_BI")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/60
46/46 [==============================] - 6s 64ms/step - loss: 1.3740 - accuracy: 0.3174 - f1: 0.0014 - precision: 0.0465 - recall: 7.3168e-04 - val_loss: 1.2914 - val_accuracy: 0.4075 - val_f1: 0.0357 - val_precision: 0.8611 - val_recall: 0.0183
Epoch 2/60
46/46 [==============================] - 2s 43ms/step - loss: 1.2099 - accuracy: 0.4714 - f1: 0.1487 - precision: 0.6206 - recall: 0.0915 - val_loss: 1.0432 - val_accuracy: 0.5567 - val_f1: 0.4596 - val_precision: 0.6752 - val_recall: 0.3492
Epoch 3/60
46/46 [==============================] - 2s 43ms/step - loss: 0.9594 - accuracy: 0.5987 - f1: 0.5160 - precision: 0.7322 - recall: 0.4008 - val_loss: 0.8475 - val_accuracy: 0.6442 - val_f1: 0.6191 - val_precision: 0.7391 - val_recall: 0.5333
Epoch 4/60
46/46 [==============================] - 2s 42ms/step - loss: 0.7085 - accuracy: 0.7284 - f1: 0.7119 - precision: 0.8197 - recall: 0.6306 - val_loss: 0.7976 - val_accuracy: 0.7008 - val_f1: 0.6952 - val_precision: 0.7380 - val

In [20]:
model_glove_rnn = tf.keras.Sequential()
model_glove_rnn.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_glove_rnn.add(tf.keras.layers.SimpleRNN(64))
model_glove_rnn.add(tf.keras.layers.Dropout(0.2))
model_glove_rnn.add(tf.keras.layers.Dense(64,activation='relu'))
model_glove_rnn.add(tf.keras.layers.Dropout(0.2))
model_glove_rnn.add(tf.keras.layers.Dense(4, activation='softmax'))
model_glove_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy',f1,precision,recall])
history_glove_rnn = model_glove_rnn.fit_generator(glove_train_gen,validation_data=glove_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_glove_rnn.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[8]=history[1]
print("GLOVE_RNN")

Epoch 1/60


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


46/46 [==============================] - 5s 89ms/step - loss: 1.3896 - accuracy: 0.2616 - f1: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.3708 - val_accuracy: 0.3317 - val_f1: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/60
46/46 [==============================] - 4s 82ms/step - loss: 1.3803 - accuracy: 0.3064 - f1: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.3712 - val_accuracy: 0.3283 - val_f1: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/60
46/46 [==============================] - 4s 85ms/step - loss: 1.3746 - accuracy: 0.3113 - f1: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.3583 - val_accuracy: 0.3367 - val_f1: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/60
46/46 [==============================] - 4s 80ms/step - loss: 1.3474 - accuracy: 0.3532 - f1: 0.0159 - precision: 0.1600 - recall: 0.0088 - val_loss: 1.3090 - val_accura

In [21]:
print("\tLstm\t\t\tBiLstm\t\t\tRnn")
print("sg",result_table[:3])
print("cbow",result_table[3:6])
print("glove",result_table[6:9])
# print(history_sg_bi.history)
# plt.title("Skip-Gram + Bi-LSTM accuracy, loss vs epochs Graph")
# plt.plot(history_sg_bi.history['loss'],c='b',label='loss')
# plt.plot(history_sg_bi.history['accuracy'],c='r',label='accuracy')
# plt.xlabel('epochs')
# plt.legend()
# # plt.yticks()
# plt.show()
# plt.title("GloVe + Bi-LSTM accuracy, loss vs epochs Graph")
# plt.plot(history_glove_bi.history['loss'],c='b',label='loss')
# plt.plot(history_glove_bi.history['accuracy'],c='r',label='accuracy')
# plt.xlabel('epochs')
# plt.legend()
# # plt.yticks()
# plt.show()

	Lstm			BiLstm			Rnn
sg [0.6000000238418579, 0.6142857074737549, 0.2978571355342865]
cbow [0.5699999928474426, 0.631428599357605, 0.3378571569919586]
glove [0.7257142663002014, 0.7599999904632568, 0.386428564786911]
